In [124]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
import json
import urllib.request
import numpy as np
import time
import dash
import dash_core_components as dcc
import dash_html_components as html
import colorlover as cl
from IPython.display import HTML

#authenticate plotly:
plotly.tools.set_credentials_file(username='smorabito', api_key='BoBmLuy3saDfBmpvhsM7')

#necessary for plotly offline:
init_notebook_mode(connected=True)

In [5]:
items = [[0,0], [1,1]]

a = {"items": items}
print("a", a.values())
print("items:", items, "\n")

items[0][0] += 1

print("a:", a.values())
print("items:", items, "\n")

items = []

print("a:", a.values())
print("items", items)

a dict_values([[[0, 0], [1, 1]]])
items: [[0, 0], [1, 1]] 

a: dict_values([[[1, 0], [1, 1]]])
items: [[1, 0], [1, 1]] 

a: dict_values([[[1, 0], [1, 1]]])
items []


In [6]:
tr = go.Scatter(
    x = [1,1,1,1,1],
    y = [0,1,2,3,4],
    text = ["test " + str(i) for i in range(10)],
    mode = 'markers+text',
    name = 'markers',
    hoverinfo='text',
    hovertext = ["test " + str(i) for i in range(10)],
    textposition = 'bottom',
    hoveron = 'points'
)

layout = go.Layout(
    xaxis = dict(
        ticks='',
        zeroline=False,
        showticklabels=False
    ),
    yaxis = dict(
        zeroline=False,
        showline=False,
        ticks='',
        showticklabels=False
    ),
    showlegend=False,
)

fig = go.Figure(data=[tr], layout=layout)
iplot(fig)

In [156]:
#color interpolation
c = cl.scales['11']['qual']['Paired']
HTML( cl.to_html(c) )

In [158]:
bupu500 = cl.interp( c, 30 ) # Map color scale to 500 bins
#print(bupu500)
HTML( cl.to_html( bupu500 ) )

In [184]:
#start dealing with actual JSON to build these graphs:

def rsid_url(rsid):
    return "https://www.t2depigenome.org/peak_metadata/region=%s&genome=GRCh37/peak_metadata.tsv" % rsid

In [185]:
#remake generate shapes to only draw one shape around all annnotations with a common biosample
def generate_shapes(positions, var_name, biosample_colors):
    
    shape_coords = [positions[var_name]["shape-coords"]]
    shape_biosamples = [var_name]
    shapes = []
    
    for anno in positions[var_name]["annotations"].keys():
        shape_coords.append(positions[var_name]["annotations"][anno]["shape-coords"])
        shape_biosamples.append(anno)
        last_biosample = ""
        for item in positions[var_name]["annotations"][anno]["items"]:
            cur_biosample = item["biosample_term_name"]
            if cur_biosample != last_biosample:
                shape_coords.append(item["shape-coords"])
                shape_biosamples.append(item["biosample_term_name"])
            last_biosample = cur_biosample
    
    #make shape color list:
    colors = []
    for i, biosample in enumerate(shape_biosamples):
        if biosample in biosample_colors.keys():
            colors.append(biosample_colors[biosample])
        else:
            colors.append('rgba(55, 128, 191, 0.6)')
    print("colors:", len(colors))
    print("shape coords:", len(shape_coords))
    print("shape biosamples:", len(shape_biosamples))
    
    for i, coords in enumerate(shape_coords):
        shapes.append({
            'type': 'rect',
            'xref': 'x',
            'yref': 'y',
            'x0': coords[0][0],
            'y0': coords[0][1],
            'x1': coords[1][0],
            'y1': coords[1][1],
            'line': {
                'color': '#888',
                'width': 2,
            },
            #'fillcolor': 'rgba(55, 128, 191, 0.6)',
            'fillcolor': colors[i],
            'opacity': 0.3
        })
    return shapes

In [186]:
#new generate positions grouping annotation in the same biosample in one box
def generate_positions(var_data, var_name, vert_space=4, box_width=10, \
                       box_height=4, text_y0=100, plot_width=100, offset=20, expanded=True, biosamples=""):
    
    #sort items within var_data (should I do this outside of this function??):
    var_data = {anno: sorted(items, key=lambda x: x['biosample_term_name']) for anno, items in var_data.items()}
    
    #get number of annotations
    num_annotations = len(var_data.keys())
    num_sides = int(num_annotations/2)   
    
    positions = {
        var_name: {
            "text-coords": [plot_width/2, text_y0],
            "shape-coords": [
                [(plot_width/2)-box_width/2, text_y0-box_height/2],
                [(plot_width/2)+box_width/2, text_y0+box_height/2]
            ],
            "annotations": {key: {"items": val} for key, val in var_data.items()}
        }
    }
    
    for i, anno in enumerate(positions[var_name]["annotations"].keys()):
        #get x offset:
        if num_annotations % 2 == 1 and i == int(num_annotations/2)+1:
            x_offset = positions[var_name]["text-coords"][0] + offset
        elif i+1 <= num_sides: 
            x_offset = positions[var_name]["text-coords"][0] - offset*(i+1)
        elif i+1 > num_sides and num_annotations % 2 == 1:
            x_offset = positions[var_name]["text-coords"][0] + offset*(i-int(num_annotations/2))
        else:
            x_offset = positions[var_name]["text-coords"][0] + offset*((i+1)-int(num_annotations/2))
            
        #set text coordinates for this annotation
        positions[var_name]["annotations"][anno]["text-coords"] = [
            x_offset, 
            text_y0-(2*vert_space)
        ]
        
        #set shape coordinates for this annotation
        positions[var_name]["annotations"][anno]["shape-coords"] = [
            [
                positions[var_name]["annotations"][anno]["text-coords"][0]-box_width/2,
                positions[var_name]["annotations"][anno]["text-coords"][1]-box_height/2
            ],
            [
                positions[var_name]["annotations"][anno]["text-coords"][0]+box_width/2,
                positions[var_name]["annotations"][anno]["text-coords"][1]+box_height/2
            ]
        ]
        
        cur_text_coords = []
        cur_shape_coords = []
        last_biosample = ""

        if expanded == True:
            for j, item in enumerate(positions[var_name]["annotations"][anno]["items"]):
                cur_biosample = item["biosample_term_name"]
                if cur_biosample in biosamples:
                    #text coordinates
                    if len(cur_text_coords) == 0:
                        item["text-coords"] = [
                            positions[var_name]["annotations"][anno]["text-coords"][0],
                            positions[var_name]["annotations"][anno]["text-coords"][1] - box_height
                        ]
                        cur_text_coords.append(item["text-coords"])
                    else:
                        item['text-coords'] = [
                            cur_text_coords[-1][0],
                            cur_text_coords[-1][1] - box_height
                        ]
                        cur_text_coords.append(item["text-coords"])
                    #shape coordinates (make a new shape or extend last shape):
                    #make a new shape
                    if j == 0 or cur_biosample != last_biosample:
                        cur_shape_coords = [
                            [item["text-coords"][0]-box_width/2, item["text-coords"][1]-box_height/2],
                            [item["text-coords"][0]+box_width/2, item["text-coords"][1]+box_height/2]
                        ]                        
                    #extend y value of current shape
                    elif cur_biosample == last_biosample:
                        cur_shape_coords[0][1] -= box_height
                    item["shape-coords"] = cur_shape_coords
                    
                    #set last biosample to cur biosample
                    last_biosample = cur_biosample           
        
    return positions

In [187]:
def make_graph(var_data, var_name, vert_space=4, box_width=10, \
                box_height=4, text_y0=100, plot_width=1000, plot_height=500, offset=20, expanded=True, biosamples=""):
    
    print("biosamples:", biosamples)
    if biosamples == "":
        biosamples = get_biosamples(var_data)
    
    #get max number of items in one annotation (to compute plot height)
    max_items = max([len([item for item in var_data[anno] if item["biosample_term_name"] in biosamples]) for anno in var_data.keys()])
    num_annotations = len(var_data.keys())
    plot_height = 350*np.log(max_items)
    plot_width = 900 * np.log(num_annotations)
 
    #longest label
    box_width = max_text_len(var_data)
    offset = box_width + 5

    positions = generate_positions(var_data, var_name, expanded=expanded, box_height=box_height, \
                                   box_width=box_width, offset=offset, text_y0=text_y0, biosamples=biosamples)
    
    #initialize node and edge traces:
    node_trace = go.Scatter(
        x = [],
        y = [],
        text = [],
        mode='markers+text',
        hoverinfo='closest',
        marker=dict(
            color=[],
            size=10,
            line=dict(width=10),
            opacity=0.0
        ),
        textfont=dict(
            color='black',
            size=18,
            family='Arail',
        )
    )

    edge_trace = go.Scatter(
        x = [],
        y = [],
        line=dict(width=2,color='#888'),
        hoverinfo='none',
        mode='lines'        
    )
    
    #populate node information with text positions:
    
    node_trace['x'].append(positions[var_name]["text-coords"][0])
    node_trace['y'].append(positions[var_name]["text-coords"][1])
    node_trace['text'].append(var_name)
    for anno in positions[var_name]["annotations"].keys():
        node_trace['x'].append(positions[var_name]["annotations"][anno]["text-coords"][0])
        node_trace['y'].append(positions[var_name]["annotations"][anno]["text-coords"][1])
        node_trace['text'].append(anno)
        last_biosample = ""
        for item in positions[var_name]["annotations"][anno]["items"]:
            if item["biosample_term_name"] in biosamples:
                cur_biosample = item["biosample_term_name"]
                node_trace['x'].append(item["text-coords"][0])
                node_trace['y'].append(item["text-coords"][1])
                if cur_biosample != last_biosample:
                    text = item["biosample_term_name"] + ":<br>"
                else: 
                    text = ""
                if "state" in item.keys():
                    text += item["state"]
                elif "footprint" in item.keys():
                    text += item["footprint"]
                elif "target" in item.keys():
                    text += item['target']
                node_trace["text"].append(text)
                last_biosample = cur_biosample

            
    #fill in edges between variant name and annotation names:
    for anno in positions[var_name]["annotations"].keys():
        edge_trace['x'] += [positions[var_name]["text-coords"][0], positions[var_name]["annotations"][anno]["text-coords"][0], None]
        edge_trace['y'] += [positions[var_name]["shape-coords"][0][1], positions[var_name]["annotations"][anno]["shape-coords"][1][1], None]
    
    #add invisible points:
    min_coords, max_coords = invisible_points(positions, var_name)
    node_trace['x'].append(min_coords[0])
    node_trace['y'].append(min_coords[1])
    node_trace['x'].append(max_coords[0])
    node_trace['y'].append(max_coords[1])
    
    layout = go.Layout(
        width=plot_width,
        height=plot_height,
        xaxis = dict(
            showgrid=False, 
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False
        ),
        yaxis = dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False
        ),
        showlegend=False,        
    )
    
    biosample_colors = get_biosample_colors(biosamples)
    layout['shapes'] = generate_shapes(positions, var_name, biosample_colors)

    data = [node_trace, edge_trace]
    fig = go.Figure(data=data, layout=layout)
    return fig
    

In [188]:
#finds the longest label name
def max_text_len(var_data):
    
    max_text_len = 0
    
    for anno, items in var_data.items():
        if len(anno) > max_text_len:
            max_text_len = len(anno)
        for item in items:
            text = item["biosample_term_name"] + ": "
            if "state" in item.keys():
                text += "state"
            elif "target" in item.keys():
                text += "target"
            elif "footprint" in item.keys():
                text += "footprint"
            if len(text) > max_text_len:
                max_text_len = len(text)
    
    return max_text_len

In [189]:
def invisible_points(positions, var_name):
    
    min_coords = [i for i in positions[var_name]["text-coords"]]
    max_coords = [i for i in positions[var_name]["text-coords"]]
    
    for anno, info in positions[var_name]["annotations"].items():
        if info["text-coords"][0] > max_coords[0]:
            max_coords[0] = info["text-coords"][0]
        elif info["text-coords"][0] < min_coords[0]:
            min_coords[0] = info["text-coords"][0]
        if info["text-coords"][1] > max_coords[1]:
            max_coords[1] = info["text-coords"][1]
        elif info["text-coords"][1] < min_coords[1]:
            min_coords[1] = info["text-coords"][1]
        
        for item in positions[var_name]["annotations"][anno]["items"]:
            if "text-coords" in item.keys():
                if item["text-coords"][0] > max_coords[0]:
                    max_coords[0] = item["text-coords"][0]
                elif item["text-coords"][0] < min_coords[0]:
                    min_coords[0] = item["text-coords"][0]
                if item["text-coords"][1] > max_coords[1]:
                    max_coords[1] = item["text-coords"][1]
                elif item["text-coords"][1] < min_coords[1]:
                    min_coords[1] = item["text-coords"][1]
        
    #extend invisible x points to avoid text cutoffs:
    min_coords[0] -= 25
    max_coords[0] += 25
    
    return min_coords, max_coords

In [190]:
#get all biosample_term_names in var_data:
def get_biosamples(var_data):
    biosamples = []
    for anno in var_data.keys():
        for item in var_data[anno]:
            biosamples.append(item["biosample_term_name"])

    return list(set(biosamples))

In [202]:
#get a unique random color for each biosample:
def get_biosample_colors(biosamples):
    print("get_biosample_colors", len(biosamples)*3)
    c = cl.scales['11']['qual']['Paired']
    c_ = cl.interp(c, len(biosamples)*3)
    return {biosample: c_[i] for i, biosample in enumerate(biosamples)}    

In [211]:
c = cl.scales['11']['qual']['Paired']
c_ = cl.interp(c, 57)

In [204]:
thing = json.loads(open("../testing_files/testing.json").read())
#iplot(make_graph(var_data=thing, var_name="test",box_height=4, biosamples=["skeletal muscle tissue"]))
iplot(make_graph(var_data=thing, var_name="test",box_height=4))

biosamples: 
get_biosample_colors 51


IndexError: list index out of range

In [33]:
thing = json.loads(open("../testing_files/testing.json").read())
thing = {anno: sorted(items, key=lambda x: x['biosample_term_name']) for anno, items in thing.items()}